# Proyecto Data Science 1 copas del mundo

In [5]:
import pandas as pd                             #importar pandas para usar DataFrames
import pickle                                   #
from string import ascii_uppercase as alphabet  #convertir el formato del texto



## Scrapping Sitio web


In [6]:
# extracting all tables in website
all_tables = pd.read_html('https://en.wikipedia.org/wiki/2022_FIFA_World_Cup')
all_tables[11]            #la tabla A no tiene la misma forma que las demas (omitimos)

,Pos,Team
0,C1,Argentina
1,C2,Saudi Arabia
2,C3,Mexico
3,C4,Poland


##Limpiar Tabla (Cambiar nombre a columna Team y eliminar Qualification)

In [7]:
# Tablas desde la A a la H
# segmentos de 7 en 7 para encontrar cada letra
all_tables = pd.read_html('https://en.wikipedia.org/wiki/2022_FIFA_World_Cup')
for i in range(18,67,7): # 11 18 25
    #print(i)
    df = all_tables[i]                                      #cada un a de las tablas
    df.rename(columns={df.columns[1]:'Team'}, inplace=True) #renombrar columna 1 y actulaizar cambios (inplace)
    df.pop('Qualification')                                 #eliminar columna Qualification
    #print(df)                                              # tabla
print(alphabet)                                             #alfabeto

ABCDEFGHIJKLMNOPQRSTUVWXYZ


##como aplicar for zip con el alfabeto y el numero de grupos

In [8]:
for letra,i in zip (alphabet, range(11,67,7)):
    print(letra, i)                                 #zip crea una relacion 1 a 1 de dos list

A 11
B 18
C 25
D 32
E 39
F 46
G 53
H 60


##Crear una Tabla para cada letra desde B, Group_B - Group_H

In [12]:
all_tables = pd.read_html('https://en.wikipedia.org/wiki/2022_FIFA_World_Cup')
list_alfabeto=['B','C','D','E','F','G','H']

dict_table = {}
for letter, i in zip(list_alfabeto, range(18,67,7)): # A=11, B=18, ...
    df = all_tables[i]
    df.rename(columns={df.columns[1]:'Team'}, inplace=True)
    df.pop('Qualification')
    dict_table[f'Group {letter}'] = df
    #print(df)
dict_table['Group B']

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,1,Netherlands,3,2,1,0,5,1,+4,7
1,2,Senegal,3,2,0,1,5,4,+1,6
2,3,Ecuador,3,1,1,1,4,3,+1,4
3,4,Qatar (H),3,0,0,3,1,7,−6,0


##Guardar tablas en el directorio

In [15]:
# Upload (..verify if uploaded correctly)
with open('dict_table', 'wb') as output:
    pickle.dump(dict_table, output)


##Mundiales jugados desde 1930-2018


In [39]:
import pandas as pd                             #importar pandas
from bs4 import BeautifulSoup                   #Beatyfulsoap -para  el scraping en xml
import requests                                 #request - para hacer peticiones

years = [1930, 1934, 1938, 1950, 1954, 1958, 1962, 1966, 1970, 1974,
         1978, 1982, 1986, 1990, 1994, 1998, 2002, 2006, 2010, 2014,
         2018]


def get_matches(year):
    web = f'https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup'
    response = requests.get(web)
    content = response.text
    soup = BeautifulSoup(content, 'lxml')
    matches = soup.find_all('div', class_='footballbox')

    home = []
    score = []
    away = []

    for match in matches:
        home.append(match.find('th', class_='fhome').get_text())
        score.append(match.find('th', class_='fscore').get_text())
        away.append(match.find('th', class_='faway').get_text())

    dict_football = {'home': home, 'score': score, 'away': away}
    df_football = pd.DataFrame(dict_football)
    df_football['year'] = year
    return df_football
#get_matches(2014)                       # Mundial por año





,home,score,away,year
0,Brazil,3–1,Croatia,2014
1,Mexico,1–0,Cameroon,2014
2,Brazil,0–0,Mexico,2014
3,Cameroon,0–4,Croatia,2014
4,Cameroon,1–4,Brazil,2014
...,...,...,...,...
59,Netherlands,0–0 (a.e.t.),Costa Rica,2014
60,Brazil,1–7,Germany,2014
61,Netherlands,0–0 (a.e.t.),Argentina,2014
62,Brazil,0–3,Netherlands,2014


##Todos los años

In [41]:
# results: historical data
fifa = [get_matches(year) for year in years]
df_fifa = pd.concat(fifa, ignore_index=True)
df_fifa.to_csv("fifa_worldcup_historical_data.csv", index=False)

# fixture
df_fixture = get_matches(2022)
df_fixture.to_csv('fifa_worldcup_fixture.csv', index=False)